<a href="https://colab.research.google.com/github/kaczquszka/kwiatuszki/blob/main/model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from datasets import Dataset, DatasetDict
dataset = pd.read_csv("answers_tab.csv", sep ="\t")

In [14]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [4]:
id2label = {0:'negative', 1:'neutral', 2:'postive'}
label2id = {'negative': 0, 'neutral':1, 'positive':2}
model_checkpoint = 'distilbert-base-uncased'
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels = 3, id2label = id2label, label2id=label2id)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
dataset = Dataset.from_dict({'text': dataset.iloc[:,0], 'label':dataset.iloc[:,-1]})
dataset = DatasetDict({'train': dataset})

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1061
    })
})

In [7]:
from collections import Counter

Counter(dataset['train']["label"])

Counter({'positive': 388, 'negative': 352, 'neutral': 321})

In [8]:
def mapping(example):
  example['label'] = label2id[example['label']]
  return example

In [9]:
dataset = dataset['train'].map(mapping)

Map:   0%|          | 0/1061 [00:00<?, ? examples/s]

In [10]:
Counter(dataset['label'])

Counter({2: 388, 0: 352, 1: 321})

In [11]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 1061
})

In [12]:
from sklearn.model_selection import train_test_split
dataset = dataset.train_test_split( test_size = 0.2)
#20% do testu

In [13]:
dataset
#https://www.learnhuggingface.com/notebooks/hugging_face_text_classification_tutorial?

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 848
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 213
    })
})

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [57]:
tokenizer('no i dont like it')
tokenizer.model_max_length = 100 #tylko 100 bo mamy krótkie zdania, być może trzeba będzie rozważyć wydłużenie
tokenizer.convert_ids_to_tokens(tokenizer('Marta').input_ids)

['[CLS]', 'marta', '[SEP]']

In [58]:
def tokenize_datset(example):
  return tokenizer(example['text'], truncation=True, padding=True)

In [59]:
tokenized_dataset = dataset.map(tokenize_datset, batched =True, batch_size=1000)

Map:   0%|          | 0/848 [00:00<?, ? examples/s]

Map:   0%|          | 0/213 [00:00<?, ? examples/s]

In [63]:
tokenized_dataset.keys()

dict_keys(['train', 'test'])

In [82]:
import evaluate
import numpy as np
accuracy = evaluate.load('accuracy')

In [83]:
def compute_accuracy(predictions_and_labels: tuple[np.array, np.array]):
  predictions, labels = predictions_and_labels
  if (len(predictions.shape)>=2):
    predictions = np.argmax(predictions, axis=1)
  return accuracy.compute(predictions=predictions, references=labels)

In [85]:
# Create example list of predictions and labels
example_predictions_all_correct = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
example_predictions_one_wrong = np.array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
example_labels = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# Test the function
print(f"Accuracy when all predictions are correct: {compute_accuracy((example_predictions_all_correct, example_labels))}")
print(f"Accuracy when one prediction is wrong: {compute_accuracy((example_predictions_one_wrong, example_labels))}")

Accuracy when all predictions are correct: {'accuracy': 1.0}
Accuracy when one prediction is wrong: {'accuracy': 0.9}
